## Анализ и визуализация данных на языке Python
### Библиотека pandas. Работа с датасетами.

- Загрузка датасетов
- Обработка отсутствующих данных
- Поиск и удаление дублей
- Создание новых признаков, функции ```apply()``` и ```map()```


In [1]:
# загрузите необходимые библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
np.set_printoptions(linewidth=110, legacy="1.25")
# Настройки для pandas'a
# pd.set_option()

### Загрузка датасетов

Pandas поддерживает загрузку данных из множества источников.   
Чаще всего придется работать с данными в форматах CSV, XLSX и JSON, а также загружать их из базы данных.

Рассмотрим загрузку данных из файла формата csv - данных, разделенных запятыми.  
Посмотрим содержимое файла, который мы будем загружать:

In [3]:
with open('data/load_example1.csv') as f:
    print(f.read())

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo
16,,18,18,bar
9,10,11,12,baz
1,2,3,4,hello
0,1,,,xxx
7,6,5,,yyy


Для загрузки будем использовать функцию ```pd.read_csv()```. Укажите в качестве параметра имя файла.

In [4]:
df = pd.read_csv('data/load_example1.csv')
df

,a,b,c,d,message
0,1,2.0,3.0,4.0,hello
1,5,6.0,7.0,8.0,world
2,9,10.0,11.0,12.0,foo
3,16,NaN,18.0,18.0,bar
4,9,10.0,11.0,12.0,baz
5,1,2.0,3.0,4.0,hello
6,0,1.0,NaN,NaN,xxx
7,7,6.0,5.0,NaN,yyy


Обратите внимание, как ведет себя функция по умолчанию:
 - названия колонок соответствуют содержимому первой строки файла
 - индекс по умолчанию - последовательность чисел.

Чтобы ```read_csv()``` включила первую строку в наш DataFrame, передайте ей параметр ```header=None```:

In [5]:
df = pd.read_csv('data/load_example1.csv', header=None)
df

,0,1,2,3,4
0,a,b,c,d,message
1,1,2,3,4,hello
2,5,6,7,8,world
3,9,10,11,12,foo
4,16,NaN,18,18,bar
5,9,10,11,12,baz
6,1,2,3,4,hello
7,0,1,NaN,NaN,xxx
8,7,6,5,NaN,yyy


Также можно задать названия столбцов самостоятельно:

In [6]:
df = pd.read_csv(
    'data/load_example1.csv',
    names=['aa', 'bb', 'cc', 'dd', 'mmessage']
)
df
# можно назначить индексом любую колонку с помощью метода .set_index(column_name)
df.set_index("aa")
df

,aa,bb,cc,dd,mmessage
0,a,b,c,d,message
1,1,2,3,4,hello
2,5,6,7,8,world
3,9,10,11,12,foo
4,16,NaN,18,18,bar
5,9,10,11,12,baz
6,1,2,3,4,hello
7,0,1,NaN,NaN,xxx
8,7,6,5,NaN,yyy


,bb,cc,dd,mmessage
aa,,,,
a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo
16,NaN,18,18,bar
9,10,11,12,baz
1,2,3,4,hello
0,1,NaN,NaN,xxx
7,6,5,NaN,yyy


,aa,bb,cc,dd,mmessage
0,a,b,c,d,message
1,1,2,3,4,hello
2,5,6,7,8,world
3,9,10,11,12,foo
4,16,NaN,18,18,bar
5,9,10,11,12,baz
6,1,2,3,4,hello
7,0,1,NaN,NaN,xxx
8,7,6,5,NaN,yyy


Чтобы указать, что один из столбцов - индекс, используйте параметр `index_col`, там можно указать либо название поля, либо его порядковый номер:

In [7]:
df = pd.read_csv('data/load_example1.csv', index_col='message')
df

,a,b,c,d
message,,,,
hello,1,2.0,3.0,4.0
world,5,6.0,7.0,8.0
foo,9,10.0,11.0,12.0
bar,16,NaN,18.0,18.0
baz,9,10.0,11.0,12.0
hello,1,2.0,3.0,4.0
xxx,0,1.0,NaN,NaN
yyy,7,6.0,5.0,NaN


In [8]:
df.index
df.index.name

Index(['hello', 'world', 'foo', 'bar', 'baz', 'hello', 'xxx', 'yyy'], dtype='object', name='message')

'message'

In [9]:
# или по индексу столбца
df = pd.read_csv('data/load_example1.csv', index_col=4)
df.columns.name = 'Codes'
df

Codes,a,b,c,d
message,,,,
hello,1,2.0,3.0,4.0
world,5,6.0,7.0,8.0
foo,9,10.0,11.0,12.0
bar,16,NaN,18.0,18.0
baz,9,10.0,11.0,12.0
hello,1,2.0,3.0,4.0
xxx,0,1.0,NaN,NaN
yyy,7,6.0,5.0,NaN


In [10]:
# или
df = pd.read_csv('data/load_example1.csv', index_col=3)
df
df.loc[np.nan]

,a,b,c,message
d,,,,
4.0,1,2.0,3.0,hello
8.0,5,6.0,7.0,world
12.0,9,10.0,11.0,foo
18.0,16,NaN,18.0,bar
12.0,9,10.0,11.0,baz
4.0,1,2.0,3.0,hello
NaN,0,1.0,NaN,xxx
NaN,7,6.0,5.0,yyy


,a,b,c,message
d,,,,
NaN,0,1.0,NaN,xxx
NaN,7,6.0,5.0,yyy


Чтобы пропустить те или иные строки, используйте параметр ```skiprows```, ему можно передать список строк, которые надо пропустить:

In [11]:
df = pd.read_csv('data/load_example1.csv', skiprows=[0,1])
df

,5,6,7,8,world
0,9,10.0,11.0,12.0,foo
1,16,NaN,18.0,18.0,bar
2,9,10.0,11.0,12.0,baz
3,1,2.0,3.0,4.0,hello
4,0,1.0,NaN,NaN,xxx
5,7,6.0,5.0,NaN,yyy


Чтобы переименовать колонки используем функцию `.rename()`

In [12]:
# https://stackoverflow.com/questions/11346283/renaming-column-names-in-pandas
df.columns
# columns - это словарь, где ключ - старое имя, значение - новое
df.rename(columns={'5': 'first'}, inplace=True)
df.columns
df

Index(['5', '6', '7', '8', 'world'], dtype='object')

Index(['first', '6', '7', '8', 'world'], dtype='object')

,first,6,7,8,world
0,9,10.0,11.0,12.0,foo
1,16,NaN,18.0,18.0,bar
2,9,10.0,11.0,12.0,baz
3,1,2.0,3.0,4.0,hello
4,0,1.0,NaN,NaN,xxx
5,7,6.0,5.0,NaN,yyy


Обратите внимание: указанные строки вообще не участвуют в разборе файла!

При разборе CSV-файлов также могут встретиться следующие трудности:
 - вместо отсутствующих данных могут быть строки типа "NULL", "n/a" и т.п.
 - разделителями могут быть символы ";" (особенно при выгрузке данных из русской версии Microsoft Excel), или же символ табуляции.

Со всем этим может справиться функция ```read_csv()```.  
Загрузим файл ```data/load_example2.csv```

In [13]:
with open('data/load_example2.csv', encoding='utf-8') as f:
    print(f.read())

col1;col2;col3
1;2;3
4;"данные отсутствуют";6
7;8;9
"данные отсутствуют";8;9



 Для указания символа ";" в качестве разделителя, передайте фукнции параметр ```sep=';'```

In [14]:
df = pd.read_csv('data/load_example2.csv', sep=';')
df

,col1,col2,col3
0,1,2,3
1,4,данные отсутствуют,6
2,7,8,9
3,данные отсутствуют,8,9


In [15]:
df.dtypes

,0
col1,object
col2,object
col3,int64


Для указания нескольких символов в качестве разделителя,  
передайте фукнции параметр `sep='[;:,]'`(регулярное выражение) и `engine='python'`

In [16]:
with open('data/load_example3.csv', encoding='utf-8') as f:
    print(f.read())

col1;col2;col3
1:2	3
4;"данные отсутствуют"	6
7,8;9
"данные отсутствуют";8	9



In [17]:
df1 = pd.read_csv('data/load_example3.csv', engine='python', sep='[;:,\\t]')
df1

,col1,col2,col3
0,1,2,3
1,4,"""данные отсутствуют""",6
2,7,8,9
3,"""данные отсутствуют""",8,9


Чтобы обработать строки "данные отсутствуют" в данном примере, функции ```read_csv()``` нужно передать параметр ```na_values='данные отсутствуют'```

In [18]:
df = pd.read_csv('data/load_example2.csv', sep=';', na_values='данные отсутствуют')
df
df.dtypes

,col1,col2,col3
0,1.0,2.0,3
1,4.0,NaN,6
2,7.0,8.0,9
3,NaN,8.0,9


,0
col1,float64
col2,float64
col3,int64


### Обработка отсутствующих данных

С отсутствующими данными в объекте Series можно сделать следующее:
 - удалить функцией ```.dropna()```
 - заполнить подходящим значением, используя функцию ```.fillna()```.

Для поиска пустых значений используем функцию ```.isnull()```.

Посмотрим, как это работает на примере первого сета. Снова загрузим его.

In [19]:
df = pd.read_csv('data/load_example1.csv', index_col='message')
df

,a,b,c,d
message,,,,
hello,1,2.0,3.0,4.0
world,5,6.0,7.0,8.0
foo,9,10.0,11.0,12.0
bar,16,NaN,18.0,18.0
baz,9,10.0,11.0,12.0
hello,1,2.0,3.0,4.0
xxx,0,1.0,NaN,NaN
yyy,7,6.0,5.0,NaN


Получить series из позиций в 'b', содержащих NaN, можно используя булеву маску по колонке "b":

In [20]:
df['b'].isnull()   # то же самое df['b'].isna()

,b
message,
hello,False
world,False
foo,False
bar,True
baz,False
hello,False
xxx,False
yyy,False


In [21]:
df['b'][df['b'].isnull()]  # df[Столбцы][Строки]
df.b[df.b.isnull()]

,b
message,
bar,NaN


,b
message,
bar,NaN


Посмотрим, как работает ```.dropna()``` в Series, получим колонку 'b' в виде этого объекта:

In [22]:
b = df['b'].copy()
b

,b
message,
hello,2.0
world,6.0
foo,10.0
bar,NaN
baz,10.0
hello,2.0
xxx,1.0
yyy,6.0


Вызовем ```dropna()```:

In [23]:
# Выкидываем целиком строку
bbd = b.dropna()
bbd

,b
message,
hello,2.0
world,6.0
foo,10.0
baz,10.0
hello,2.0
xxx,1.0
yyy,6.0


Заполним отсутствующие значения

In [24]:
# можно заполнить конкретным значением
bbf = b.fillna(0)
bbf

,b
message,
hello,2.0
world,6.0
foo,10.0
bar,0.0
baz,10.0
hello,2.0
xxx,1.0
yyy,6.0


In [25]:
# а можно средним по всей Series
bbf = b.fillna(b.mean())
bbf
(2 + 6 + 10 + 10 + 2 + 1 + 6) / 7

,b
message,
hello,2.000000
world,6.000000
foo,10.000000
bar,5.285714
baz,10.000000
hello,2.000000
xxx,1.000000
yyy,6.000000


5.285714285714286

In [26]:
c = df[['c']] ## Результат датафрейм
c

,c
message,
hello,3.0
world,7.0
foo,11.0
bar,18.0
baz,11.0
hello,3.0
xxx,NaN
yyy,5.0


In [27]:
c
# c.fillna(method='ffill') # forward fill, значение индекса 'hello'  # Старый способ
c.ffill()  # Новый способ

,c
message,
hello,3.0
world,7.0
foo,11.0
bar,18.0
baz,11.0
hello,3.0
xxx,NaN
yyy,5.0


,c
message,
hello,3.0
world,7.0
foo,11.0
bar,18.0
baz,11.0
hello,3.0
xxx,3.0
yyy,5.0


In [28]:
c
# c.fillna(method='bfill') # backward fill, значение индекса 'yyy'  # Это старый способ
c.bfill()  # это новый

,c
message,
hello,3.0
world,7.0
foo,11.0
bar,18.0
baz,11.0
hello,3.0
xxx,NaN
yyy,5.0


,c
message,
hello,3.0
world,7.0
foo,11.0
bar,18.0
baz,11.0
hello,3.0
xxx,5.0
yyy,5.0


In [29]:
c_copy = c.copy()
# Берем отдельное значение(скаляр) Строки -> столбцы
c_copy.at['yyy', 'c'] = np.nan
c_copy

,c
message,
hello,3.0
world,7.0
foo,11.0
bar,18.0
baz,11.0
hello,3.0
xxx,NaN
yyy,NaN


In [30]:
c_copy.fillna(method='pad')

<ipython-input-30-fc5500973a51>:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  c_copy.fillna(method='pad')


,c
message,
hello,3.0
world,7.0
foo,11.0
bar,18.0
baz,11.0
hello,3.0
xxx,3.0
yyy,3.0


In [31]:
c.fillna(method='pad')

<ipython-input-31-61fc48a60c34>:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  c.fillna(method='pad')


,c
message,
hello,3.0
world,7.0
foo,11.0
bar,18.0
baz,11.0
hello,3.0
xxx,3.0
yyy,5.0


В случае с DataFrame это работает похожим образом, только функция удаляет строки, в которых встречается хотя бы одно незаполненное значение:

In [32]:
df

,a,b,c,d
message,,,,
hello,1,2.0,3.0,4.0
world,5,6.0,7.0,8.0
foo,9,10.0,11.0,12.0
bar,16,NaN,18.0,18.0
baz,9,10.0,11.0,12.0
hello,1,2.0,3.0,4.0
xxx,0,1.0,NaN,NaN
yyy,7,6.0,5.0,NaN


In [33]:
df
df.dropna()

,a,b,c,d
message,,,,
hello,1,2.0,3.0,4.0
world,5,6.0,7.0,8.0
foo,9,10.0,11.0,12.0
bar,16,NaN,18.0,18.0
baz,9,10.0,11.0,12.0
hello,1,2.0,3.0,4.0
xxx,0,1.0,NaN,NaN
yyy,7,6.0,5.0,NaN


,a,b,c,d
message,,,,
hello,1,2.0,3.0,4.0
world,5,6.0,7.0,8.0
foo,9,10.0,11.0,12.0
baz,9,10.0,11.0,12.0
hello,1,2.0,3.0,4.0


Этой функции можно задать порог срабатывания, в зависимости от количества __заполненных значений__ в строке. Например, нам нужно оставить те строки, в которых заполнены как минимум три значения, остальные удалить:

In [34]:
df

,a,b,c,d
message,,,,
hello,1,2.0,3.0,4.0
world,5,6.0,7.0,8.0
foo,9,10.0,11.0,12.0
bar,16,NaN,18.0,18.0
baz,9,10.0,11.0,12.0
hello,1,2.0,3.0,4.0
xxx,0,1.0,NaN,NaN
yyy,7,6.0,5.0,NaN


In [35]:
df
df.dropna(thresh=3)  # Порог - 3 заполненных значения

,a,b,c,d
message,,,,
hello,1,2.0,3.0,4.0
world,5,6.0,7.0,8.0
foo,9,10.0,11.0,12.0
bar,16,NaN,18.0,18.0
baz,9,10.0,11.0,12.0
hello,1,2.0,3.0,4.0
xxx,0,1.0,NaN,NaN
yyy,7,6.0,5.0,NaN


,a,b,c,d
message,,,,
hello,1,2.0,3.0,4.0
world,5,6.0,7.0,8.0
foo,9,10.0,11.0,12.0
bar,16,NaN,18.0,18.0
baz,9,10.0,11.0,12.0
hello,1,2.0,3.0,4.0
yyy,7,6.0,5.0,NaN


Обратите внимание на строку "bar" - несмотря на незаполненную ячейку, она не попала под удаление!

Также можно заполнять отсутствующие данные числами:

In [36]:
df.fillna(100_500)

,a,b,c,d
message,,,,
hello,1,2.0,3.0,4.0
world,5,6.0,7.0,8.0
foo,9,10.0,11.0,12.0
bar,16,100500.0,18.0,18.0
baz,9,10.0,11.0,12.0
hello,1,2.0,3.0,4.0
xxx,0,1.0,100500.0,100500.0
yyy,7,6.0,5.0,100500.0


Эта функция работает и для заполнения "пробелов" горизонтальными/вертикальными агрегатными вычислениями.

In [37]:
# Среднее по столбцам
df.mean()  # df.mean(axis=0)

,0
a,6.000000
b,5.285714
c,8.285714
d,9.666667


In [38]:
df.fillna(df.mean())

,a,b,c,d
message,,,,
hello,1,2.000000,3.000000,4.000000
world,5,6.000000,7.000000,8.000000
foo,9,10.000000,11.000000,12.000000
bar,16,5.285714,18.000000,18.000000
baz,9,10.000000,11.000000,12.000000
hello,1,2.000000,3.000000,4.000000
xxx,0,1.000000,8.285714,9.666667
yyy,7,6.000000,5.000000,9.666667


In [39]:
df.mean().mean()  # Среднее по всему датафрейму

7.309523809523808

Чтобы эти функции отработали внутри самого объекта и не возвращали его копию, используйте параметр ```inplace=True```.

### Поиск и удаление дублей

Проверить, является ли уникальным индекс, можно, опросив свойство индекса ```is_unique```:

In [40]:
df.loc['hello']

,a,b,c,d
message,,,,
hello,1,2.0,3.0,4.0
hello,1,2.0,3.0,4.0


In [41]:
# Результат выполнения - bool -> True или False
df.index.is_unique

False

In [42]:
df

,a,b,c,d
message,,,,
hello,1,2.0,3.0,4.0
world,5,6.0,7.0,8.0
foo,9,10.0,11.0,12.0
bar,16,NaN,18.0,18.0
baz,9,10.0,11.0,12.0
hello,1,2.0,3.0,4.0
xxx,0,1.0,NaN,NaN
yyy,7,6.0,5.0,NaN


In [43]:
df
df.index.duplicated()  # Второй hello -> True

,a,b,c,d
message,,,,
hello,1,2.0,3.0,4.0
world,5,6.0,7.0,8.0
foo,9,10.0,11.0,12.0
bar,16,NaN,18.0,18.0
baz,9,10.0,11.0,12.0
hello,1,2.0,3.0,4.0
xxx,0,1.0,NaN,NaN
yyy,7,6.0,5.0,NaN


array([False, False, False, False, False,  True, False, False])

In [44]:
~df.index.duplicated() # ~ - это инверсия

array([ True,  True,  True,  True,  True, False,  True,  True])

Получить булеву маску для дубликатов по индексу можно, вызвав метод ```.duplicated()```. Применение отрицания этой маски вернет DataFrame без строки с дублированным индексом.

In [45]:
df.loc[~df.index.duplicated()]

,a,b,c,d
message,,,,
hello,1,2.0,3.0,4.0
world,5,6.0,7.0,8.0
foo,9,10.0,11.0,12.0
bar,16,NaN,18.0,18.0
baz,9,10.0,11.0,12.0
xxx,0,1.0,NaN,NaN
yyy,7,6.0,5.0,NaN


In [46]:
# Когда индекс уникальный, получаем числовой индекс, если нет - булев массив
df.index.get_loc("world")  # результат работы - 1
df.index.get_loc('hello')# результат работы - булев массив
df.index.get_loc('hello').nonzero()[0]  # результат работы - булев массив

1

array([ True, False, False, False, False,  True, False, False])

array([0, 5])

Тем же методом объекта DataFrame или Series можно получить булеву маску для дубликатов записей в датасете:

In [47]:
df

,a,b,c,d
message,,,,
hello,1,2.0,3.0,4.0
world,5,6.0,7.0,8.0
foo,9,10.0,11.0,12.0
bar,16,NaN,18.0,18.0
baz,9,10.0,11.0,12.0
hello,1,2.0,3.0,4.0
xxx,0,1.0,NaN,NaN
yyy,7,6.0,5.0,NaN


In [48]:
df
df.duplicated()

,a,b,c,d
message,,,,
hello,1,2.0,3.0,4.0
world,5,6.0,7.0,8.0
foo,9,10.0,11.0,12.0
bar,16,NaN,18.0,18.0
baz,9,10.0,11.0,12.0
hello,1,2.0,3.0,4.0
xxx,0,1.0,NaN,NaN
yyy,7,6.0,5.0,NaN


,0
message,
hello,False
world,False
foo,False
bar,False
baz,True
hello,True
xxx,False
yyy,False


In [49]:
df_new = df.copy()
# Обрщаение к скаляру по индексу
df_new.iat[0, 3] = 10
df_new
# Проверяем целиком строки
df_new.duplicated()

,a,b,c,d
message,,,,
hello,1,2.0,3.0,10.0
world,5,6.0,7.0,8.0
foo,9,10.0,11.0,12.0
bar,16,NaN,18.0,18.0
baz,9,10.0,11.0,12.0
hello,1,2.0,3.0,4.0
xxx,0,1.0,NaN,NaN
yyy,7,6.0,5.0,NaN


,0
message,
hello,False
world,False
foo,False
bar,False
baz,True
hello,False
xxx,False
yyy,False


Методу можно передать параметр ```keep=```, который не будет отмечать признаком True либо первый дубликат (значение first), либо последний (значение last).

In [50]:
# Обход снизу вверх, верхние будут дубликатами
df.duplicated(keep='last')

,0
message,
hello,True
world,False
foo,True
bar,False
baz,False
hello,False
xxx,False
yyy,False


Метод можно вызвать, передав ему список признаков, в которых нужно ограничиться поиском дубликатов:

In [51]:
df

,a,b,c,d
message,,,,
hello,1,2.0,3.0,4.0
world,5,6.0,7.0,8.0
foo,9,10.0,11.0,12.0
bar,16,NaN,18.0,18.0
baz,9,10.0,11.0,12.0
hello,1,2.0,3.0,4.0
xxx,0,1.0,NaN,NaN
yyy,7,6.0,5.0,NaN


In [52]:
df
df.duplicated(['d'])

,a,b,c,d
message,,,,
hello,1,2.0,3.0,4.0
world,5,6.0,7.0,8.0
foo,9,10.0,11.0,12.0
bar,16,NaN,18.0,18.0
baz,9,10.0,11.0,12.0
hello,1,2.0,3.0,4.0
xxx,0,1.0,NaN,NaN
yyy,7,6.0,5.0,NaN


,0
message,
hello,False
world,False
foo,False
bar,False
baz,True
hello,True
xxx,False
yyy,True


Удалить дубликаты можно функцией ```drop_duplicates()```. Она работает так же, как и ```duplicated()```, но она возвращает новый DataFrame без дубликатов. Ее можно вызвать с параметром `inplace`.

In [53]:
df.drop_duplicates()

,a,b,c,d
message,,,,
hello,1,2.0,3.0,4.0
world,5,6.0,7.0,8.0
foo,9,10.0,11.0,12.0
bar,16,NaN,18.0,18.0
xxx,0,1.0,NaN,NaN
yyy,7,6.0,5.0,NaN


In [54]:
df

,a,b,c,d
message,,,,
hello,1,2.0,3.0,4.0
world,5,6.0,7.0,8.0
foo,9,10.0,11.0,12.0
bar,16,NaN,18.0,18.0
baz,9,10.0,11.0,12.0
hello,1,2.0,3.0,4.0
xxx,0,1.0,NaN,NaN
yyy,7,6.0,5.0,NaN


### Создание новых признаков, функции apply() и map()

С созданием новых признаков на базе существующих данных мы уже знакомы, но часто бывает так, что для вычисления новых признаков нужно применить более сложные процедуры, чем стандартные. Для этого существуют функции ```apply()``` и ```map()```.



In [55]:
np.random.seed(555)
df = pd.DataFrame(np.random.randint(1, 20, size=(4, 3)), columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])
df

,b,d,e
Utah,15,10,2
Ohio,5,7,10
Texas,17,1,11
Oregon,17,5,2


Посмотрим, как работает метод ```apply()```. Функция, которая указана в качестве параметра этого метода принимает на вход объект Series - столбец и возвращает значение, которое объединяется в объект Series, структyрно соответствующий строке текущего DataFrame. Для вычисления по строкам и формирования столбцов функции ```apply()``` нужно передать параметр ```axis=1```

In [56]:
df

,b,d,e
Utah,15,10,2
Ohio,5,7,10
Texas,17,1,11
Oregon,17,5,2


In [57]:
# Пример создания и использования анонимных функций
func = lambda a, b: 2 * a + b * a - b ** 0.5
print(func(8, 5))
print((lambda a, b, c: (a + b + c) / 3)(10, 20, 30))

53.763932022500214
20.0


In [58]:
df
# Функции f и ff эквивалентны:
def f(x):
    """ x - это Series, столбец(axis=0), строка(axis=1)"""
    # print(x)
    return x.max() - x.min()

# В качестве примера альтернативного варианта. Результат тот же
ff = lambda x: x.max() - x.min()

print(df.apply(f, axis=1)) # Применяем функцию f к каждому штату(индекс)
print('************* The same **************')
print(df.apply(lambda : x.max() - x.min(), axis=1))

,b,d,e
Utah,15,10,2
Ohio,5,7,10
Texas,17,1,11
Oregon,17,5,2


Utah      13
Ohio       5
Texas     16
Oregon    15
dtype: int64
************* The same **************


TypeError: <lambda>() takes 0 positional arguments but 1 was given

In [59]:
df
df.apply(ff) # Применяем функцию f к каждому столбцу

,b,d,e
Utah,15,10,2
Ohio,5,7,10
Texas,17,1,11
Oregon,17,5,2


,0
b,12
d,9
e,9


Добавление нового вычисленного признака теперь будет выглядеть так:

In [60]:
df['diff'] = df.apply(f, axis=1)
df

,b,d,e,diff
Utah,15,10,2,13
Ohio,5,7,10,5
Texas,17,1,11,16
Oregon,17,5,2,15


В отличие от ```apply()```, ```map()``` вычисляется для каждого элемента и возвращает значение, которое должно быть установлено на его место.

In [61]:
format_ = lambda x: f'{x:.2f}'
# df.applymap(format_)  # Это старый способ
df.map(format_)  # Это новый способ (Use DataFrame.map instead)

,b,d,e,diff
Utah,15.00,10.00,2.00,13.00
Ohio,5.00,7.00,10.00,5.00
Texas,17.00,1.00,11.00,16.00
Oregon,17.00,5.00,2.00,15.00


In [62]:
df
df.map(lambda x: x ** 2 if x > 10 else x / 100) # x - это каждое значение в DataFrame

,b,d,e,diff
Utah,15,10,2,13
Ohio,5,7,10,5
Texas,17,1,11,16
Oregon,17,5,2,15


,b,d,e,diff
Utah,225.00,0.10,0.02,169.00
Ohio,0.05,0.07,0.10,0.05
Texas,289.00,0.01,121.00,256.00
Oregon,289.00,0.05,0.02,225.00


Для того, чтобы проделать такую операцию над Series, воспользуйтесь функцией `map()`:

In [63]:
df

,b,d,e,diff
Utah,15,10,2,13
Ohio,5,7,10,5
Texas,17,1,11,16
Oregon,17,5,2,15


In [64]:
df['e'] = df['e'].map(format_)
df

,b,d,e,diff
Utah,15,10,2.00,13
Ohio,5,7,10.00,5
Texas,17,1,11.00,16
Oregon,17,5,2.00,15


In [65]:
# Создаю новую колонку
df['f'] = df['e'].astype(float).map(lambda x: x ** 2 + 3 * x + 1)
df

,b,d,e,diff,f
Utah,15,10,2.00,13,11.0
Ohio,5,7,10.00,5,131.0
Texas,17,1,11.00,16,155.0
Oregon,17,5,2.00,15,11.0


In [66]:
# Выводит количество повторяющихся значений
df['b'].value_counts()

,count
b,
17,2
15,1
5,1


In [67]:
df

,b,d,e,diff,f
Utah,15,10,2.00,13,11.0
Ohio,5,7,10.00,5,131.0
Texas,17,1,11.00,16,155.0
Oregon,17,5,2.00,15,11.0


### Анализ и разбор датасета

В качестве примера рассмотрим набор данных о пассажирах затонувшего "Титаника".

In [68]:
titanic = pd.read_csv('data/titanic.csv', index_col='PassengerId')

Чтобы проверить, что у нас все загрузилось корректно, посмотрим на форму получившегося объекта:

In [69]:
titanic.shape

(891, 11)

In [70]:
titanic.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [71]:
titanic.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [72]:
titanic.Survived.value_counts()

,count
Survived,
0,549
1,342


### Задание 1

В датасете Titanic проверьте признак "Возраст"("Age") на выбросы (отрицательный возраст, посмотрите максимальный возраст - он правдоподобен?).  
Если там есть отсутствующие значения - на их место поставьте медианный возраст пассажиров.

In [73]:
# Ваш код
titanic.Age.min()
titanic.Age.max()
titanic.Age.describe()
# negative age check
negative_age = titanic[titanic['Age'] < 0]
over_age = titanic[titanic.Age > 100]
print(f"negative age exists: {not negative_age.empty}")
print(f"over age exists: {not over_age.empty}")

0.42

80.0

,Age
count,714.000000
mean,29.699118
std,14.526497
min,0.420000
25%,20.125000
50%,28.000000
75%,38.000000
max,80.000000


negative age exists: False
over age exists: False


---

### Создаем выборку обычным способом

In [74]:
titanic[(titanic.Age < 5) & (titanic.Sex == 'male')]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
8,0,3,"Palsson, Master. Gosta Leonard",male,2.00,3,1,349909,21.0750,NaN,S
17,0,3,"Rice, Master. Eugene",male,2.00,4,1,382652,29.1250,NaN,Q
64,0,3,"Skoog, Master. Harald",male,4.00,3,2,347088,27.9000,NaN,S
79,1,2,"Caldwell, Master. Alden Gates",male,0.83,0,2,248738,29.0000,NaN,S
165,0,3,"Panula, Master. Eino Viljami",male,1.00,4,1,3101295,39.6875,NaN,S
172,0,3,"Rice, Master. Arthur",male,4.00,4,1,382652,29.1250,NaN,Q
184,1,2,"Becker, Master. Richard F",male,1.00,2,1,230136,39.0000,F4,S
194,1,2,"Navratil, Master. Michel M",male,3.00,1,1,230080,26.0000,F2,S
262,1,3,"Asplund, Master. Edvin Rojj Felix",male,3.00,4,2,347077,31.3875,NaN,S


#### Примеры использования  метода: `.query()`

In [75]:
# Само выражение - это строка(str)
titanic.query("Age < 5 and Sex == 'male'")

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
8,0,3,"Palsson, Master. Gosta Leonard",male,2.00,3,1,349909,21.0750,NaN,S
17,0,3,"Rice, Master. Eugene",male,2.00,4,1,382652,29.1250,NaN,Q
64,0,3,"Skoog, Master. Harald",male,4.00,3,2,347088,27.9000,NaN,S
79,1,2,"Caldwell, Master. Alden Gates",male,0.83,0,2,248738,29.0000,NaN,S
165,0,3,"Panula, Master. Eino Viljami",male,1.00,4,1,3101295,39.6875,NaN,S
172,0,3,"Rice, Master. Arthur",male,4.00,4,1,382652,29.1250,NaN,Q
184,1,2,"Becker, Master. Richard F",male,1.00,2,1,230136,39.0000,F4,S
194,1,2,"Navratil, Master. Michel M",male,3.00,1,1,230080,26.0000,F2,S
262,1,3,"Asplund, Master. Edvin Rojj Felix",male,3.00,4,2,347077,31.3875,NaN,S


In [76]:
titanic.query("Age < 5 and Sex == 'male'")['Ticket']

,Ticket
PassengerId,
8,349909
17,382652
64,347088
79,248738
165,3101295
172,382652
184,230136
194,230080
262,347077


In [77]:
titanic.query("Age < 5 and Sex == 'male'").Fare

,Fare
PassengerId,
8,21.0750
17,29.1250
64,27.9000
79,29.0000
165,39.6875
172,29.1250
184,39.0000
194,26.0000
262,31.3875


In [78]:
value = 5

In [79]:
# Используем значение переменной в .query()
titanic.query("0 < Age < @value and Sex == 'male'")['Ticket']

,Ticket
PassengerId,
8,349909
17,382652
64,347088
79,248738
165,3101295
172,382652
184,230136
194,230080
262,347077


In [80]:
# Переводим Series в DataFrame
titanic.query(f"Age < {value} and Sex == 'male'")['Ticket'].to_frame()

,Ticket
PassengerId,
8,349909
17,382652
64,347088
79,248738
165,3101295
172,382652
184,230136
194,230080
262,347077


### Задание 2
Проанализируйте загруженные данные:

1. Сколько было на Титанике мужчин, женщин и несовершеннолетних детей(<18 лет)?
2. Действительно ли у пассажиров, путешествующих 3-м классом нет номеров кают?
3. Есть предположение, что женщин и детей не было среди путешествующих 3-м классом. Проверьте.

In [81]:
# Ваш код
# 1. Сколько было на Титанике мужчин, женщин и несовершеннолетних детей(<18 лет)?
children = titanic.query("Age < 18").index.size
men = titanic.query("Age >= 18 and Sex == 'male'").shape[0]
women = titanic.query("Age >= 18 and Sex == 'female'").Name.count()
print(f'{children = }, {men = }, {women = }')

adults = titanic.query("Age >= 18")
adults.Sex.value_counts()

children = 113, men = 395, women = 206


,count
Sex,
male,395
female,206


In [82]:
# 2. Действительно ли у пассажиров, путешествующих 3-м классом нет номеров кают?
# Пример работы с np.nan в pandas.query()
titanic.query('Pclass == 3 and Cabin == Cabin').Cabin.to_frame()
# np.nan == np.nan # False

,Cabin
PassengerId,
11,G6
76,F G73
129,F E69
206,G6
252,G6
395,G6
430,E10
700,F G63
716,F G73


In [83]:
titanic.query('Pclass == 3').Cabin.isna().all()
cabin_stats = titanic.query('Pclass == 3 & Cabin.notna()').Cabin.count()
cabin_stats

False

12

In [84]:
titanic.query('Pclass == 3').Cabin.unique()

array([nan, 'G6', 'F G73', 'F E69', 'E10', 'F G63', 'E121', 'F38'], dtype=object)

In [85]:
titanic.query('Pclass == 3').dropna().Cabin.to_frame().empty

False

In [86]:
# 3. Есть предположение, что женщин и детей не было среди путешествующих 3-м классом. Проверьте.
print("woman in 3d class", titanic.query('Pclass == 3 and Sex == "female" and Age >= 18').shape[0])
print("kids <18 in 3d class", titanic.query('Pclass == 3 and Age < 18').index.size)
print("woman or kids in 3d class", titanic.query('Pclass == 3 and (Sex == "female" | Age < 18)').shape[0])

woman in 3d class 67
kids <18 in 3d class 78
woman or kids in 3d class 187
